In [1]:
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from fastai.vision.all import URLs
from batchflow import Pipeline, B, C, D, F, V, W
from batchflow.models.torch import EncoderDecoder
from batchflow.models.torch.layers import ConvBlock
from batchflow.models.torch.layers.modules import ASPP, PyramidPooling, KSAC
from batchflow.opensets import Imagenette160

from train_module import training_functions

GRAPH_PATH = "./data/graphs/"
IMAGE_SHAPE = (3, 160, 160)

In [2]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [3]:
dataset = Imagenette160(bar='n')

 50%|█████     | 1/2 [00:05<00:05,  5.38s/it]


In [4]:
in_channels = 3
out_channels = 3
device = "cuda:3"

batch_size = 64
epoch_num = 5

In [5]:
downsample_depth = 2
in_channels = 3
config = { 
    'device': device,
    'inputs/images/shape': (3, 160, 160), # can be commented
    'initial_block/inputs': 'images', # can be commented   
    'initial_block': {
        'layout': 'cna cna',
        'strides': [2, 1],
        'filters': [32, 64]
    },

    'body/encoder': {
        'num_stages': downsample_depth,
        'order': ['downsampling', 'skip']
    },    
    'body/encoder/downsample': {
        'layout': 'R' + 'wnacna' * 3 + '|',
        'filters': 'same',
        'strides': [[1, 1, 1, 1, 2, 1]] * downsample_depth,
        'kernel_size': [[3, 1] * 3] * downsample_depth,
        'dilation_rate' : 2,
        'branch': {
            'layout': 'cn', 
            'kernel_size': 1,
            'filters': 'same',
            'strides': 2
        },
    },      

    'body/embedding': {
        'base' : ASPP
    },  

    'body/decoder': {
        'num_stages': downsample_depth,
        'order': ['block', 'combine', 'upsampling'] 
    },

    'body/decoder/blocks': {
        'layout': 'cna',
        'kernel_size': 1,
        'filters': 256
    },

    'body/decoder/upsample': {
        'layout': 'b',
        'scale_factor': 2
    },
    'body/decoder/combine': {
        'op': 'concat',
    },
    
    
    'head':{
        'layout': 'cna b',
        'scale_factor': 2,
        'filters': in_channels,
        'activation': 'sigmoid'
    },

    'loss': 'mse',
    'optimizer': 'Adam'
}

In [6]:
model = EncoderDecoder(config)